In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib widget

# SR830 Code samples

In [2]:
from pymeasure.instruments.srs import SR830

lockin = SR830('GPIB2::25::INSTR')
lockin.buffer_measure(1)


AttributeError: 'SR830' object has no attribute 'buffer_data'

# Zurich MFLI code samples

In [ ]:
import zhinst.core

In [2]:
device_id = "dev4934" # Device serial number available on its rear panel.
interface = "1GbE" # For Ethernet connection or when MFLI/MFIA is connected to a remote Data Server.
# interface = "USB" # For all instruments connected to the host computer via USB except MFLI/MFIA.
#interface = "PCIe" # For MFLI/MFIA devices in case the Data Server runs on the device.

# server_host = "localhost"
server_host = '127.0.0.1'
# server_host = "192.168.77.26"
server_port = 8004

#server_port = 8005 # Default port for HF2LI.
api_level = 6 # Maximum API level supported for all instruments except HF2LI.
#api_level = 1 # Maximum API level supported for HF2LI.

# Create an API session to the Data Server.
daq = zhinst.core.ziDAQServer(server_host, server_port, api_level)
# Establish a connection between Data Server and Device.
daq.connectDevice(device_id, interface)

In [9]:
demod_path = f"/{device_id}/demods/0/"
sample_path = demod_path+'sample'

daq.getSample(sample_path)

{'timestamp': array([227384699796142], dtype=uint64),
 'x': array([-7.57581736e-10]),
 'y': array([7.82990742e-10]),
 'frequency': array([732.9999999]),
 'phase': array([3.37437424]),
 'dio': array([0], dtype=uint32),
 'trigger': array([512], dtype=uint32),
 'auxin0': array([0.00131781]),
 'auxin1': array([0.])}

In [13]:
daq.setDouble('dev4934/oscs/0/freq', 734)



In [4]:
'/dev4934/demods/0/sample.R'

'/dev4934/demods/0/sample.R'

In [5]:
daq.getSample(demod_path+'sample')


{'timestamp': array([227374896695982], dtype=uint64),
 'x': array([1.72026712e-09]),
 'y': array([-1.18313107e-09]),
 'frequency': array([732.9999999]),
 'phase': array([2.07413377]),
 'dio': array([0], dtype=uint32),
 'trigger': array([768], dtype=uint32),
 'auxin0': array([0.00164727]),
 'auxin1': array([-0.00098523])}

In [6]:
display(daq.listNodes(demod_path))
display(daq.getSample(sample_path))

['/dev4934/demods/0/adcselect',
 '/dev4934/demods/0/order',
 '/dev4934/demods/0/rate',
 '/dev4934/demods/0/oscselect',
 '/dev4934/demods/0/harmonic',
 '/dev4934/demods/0/phaseshift',
 '/dev4934/demods/0/sinc',
 '/dev4934/demods/0/bypass',
 '/dev4934/demods/0/timeconstant',
 '/dev4934/demods/0/enable',
 '/dev4934/demods/0/freq',
 '/dev4934/demods/0/trigger',
 '/dev4934/demods/0/phaseadjust',
 '/dev4934/demods/0/sample']

{'timestamp': array([227375330610862], dtype=uint64),
 'x': array([-6.90321325e-10]),
 'y': array([-1.45805114e-09]),
 'frequency': array([732.9999999]),
 'phase': array([2.03300391]),
 'dio': array([0], dtype=uint32),
 'trigger': array([256], dtype=uint32),
 'auxin0': array([0.00098836]),
 'auxin1': array([-0.00065682])}

In [7]:
daq.getSample('/dev4934/demods/0/sample')

{'timestamp': array([227375541708462], dtype=uint64),
 'x': array([-5.79671162e-10]),
 'y': array([2.92274571e-09]),
 'frequency': array([732.9999999]),
 'phase': array([1.46130845]),
 'dio': array([0], dtype=uint32),
 'trigger': array([768], dtype=uint32),
 'auxin0': array([0.]),
 'auxin1': array([-0.00164204])}

In [8]:
# daq.setDouble(f'/{device_id}/sigouts/0/range',12E-3)



In [28]:
'{:}'.format(3)

'3'

In [12]:
import datetime as dt
dt.datetime.utcnow().timestamp()

1686521581.199947

# zi_MFLI class

In [14]:
from importlib import reload
import zurich_MFLI as zur_driver
reload(zur_driver)

daq_obj = zur_driver.zi_MFLI(device_id, interface, server_host, server_port, api_level)

In [16]:
display(daq_obj.get_XY())
display(daq_obj.get_freq())

[array([7.05867046e-06]), array([2.38378174e-07])]

array([32764.49999994])

In [21]:
daq_obj.set_freq(32764.51)
daq_obj.get_freq()

array([32764.51000005])